In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [ ]:
df_phe = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/Excels/tRNA_Phe_0724.xlsx')
# df_phe = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S2to5combined_20ul_40min.xlsx')
df_phe = thermo_df(df_phe)
df_phe.shape
plotly_zone(df_phe)

In [ ]:
df_phe = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S2to5combined_20ul_40min.xlsx')

In [ ]:
df_noad = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/oligossample1.xlsx')
df_noad = thermo_df(df_noad)
df_noad.shape
plot_zone(df_noad)

In [ ]:
df_noad_25k = df_noad[(df_noad.Mass > 20000)]
plotly_zone(df_noad_25k)

In [ ]:
bcr = base_calling_random(df_noad_25k, silence=True)
plotly_basecalling_vol(*bcr)

In [ ]:
df_phe_25k = df_phe[(df_phe.Mass > 24000) & (df_phe.Mass < 26000) & (df_phe.Vol > 1E5)]
# df_phe_25k = df_phe[(df_phe.Mass > 23900) & (df_phe.Mass < 24000)]
# df_phe_25k = df_phe[df_phe.Mass > 23000]
px.scatter(x=df_phe_25k.Mass, y=df_phe_25k.Vol)

In [ ]:
df_phe[(df_phe.Mass > 24000) & (df_phe.Vol > 500000)].sort_values('Mass')

In [ ]:
df_phe[(df_phe.Mass > 24600) & (df_phe.Mass < 25500) & (df_phe.Vol > 100000)].sort_values('Mass')

In [ ]:
bcr = base_calling_random(df_phe_25k, silence=True)
plt, _ = plot_basecalling(*bcr)
plt.savefig('/Users/bryan/Documents/Presentations/Feb 17 2021/25k_bc.png', transparent=True)

In [ ]:
plotly_basecalling(*bcr)

In [ ]:
df_5p = pd.read_excel('/Users/bryan/Downloads/phe5p.xlsx')
df_3p = pd.read_excel('/Users/bryan/Downloads/phe3p.xlsx')
df_5p.shape, df_3p.shape

In [ ]:
df_5p = df_5p[df_5p.Vol > 1E5]
df_3p = df_3p[df_3p.Vol > 1E5]
df_5p.shape, df_3p.shape

In [ ]:
full_mass = 24416.29 #24189.23 #24245.225 #24941.525 #24252.315242 #24581.374101 
df_3p_24416, df_5p_24416 = computational_data_seperation(df_3p, df_5p, full_mass, ignore_endpoints=True)
plotly_zones(df_5p_24416, df_3p_24416)
df_24416 = pd.concat([df_3p_24416, df_5p_24416]).drop_duplicates()
df_24416.shape

In [ ]:
full_mass = 24087.24 #24189.23 #24245.225 #24941.525 #24252.315242 #24581.374101 
df_3p_24087, df_5p_24087 = computational_data_seperation(df_3p, df_5p, full_mass, ignore_endpoints=True)
plotly_zones(df_5p_24087, df_3p_24087)
df_24087 = pd.concat([df_3p_24087, df_5p_24087]).drop_duplicates()
df_24087.shape

In [ ]:
tmp = df_3p_24087.copy()
tmp.Mass += 329.0525
plotly_zones(df_3p_24416, tmp)

In [ ]:
# df_merge = pd.concat([df_3p_24445, df_5p_24116]).drop_duplicates()
full_mass = 24252.32 #24581.38 #
df_skeleton = df_3p_24252 #df_5p_24597
group = 'Group_24581'

fullmass_dot = df_phe[(df_phe.Mass>full_mass-0.1) & (df_phe.Mass<full_mass+0.1)]
df_gap_dots = find_gap_dots(df_3p, df_skeleton, fullmass_dot, orientation=3)
df_gap_dots.shape

In [ ]:
plotly_zones(df_skeleton, df_gap_dots)
plt = plot_zones(df_skeleton, df_gap_dots)
plt.savefig('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/MassSum/II/Phe_MFE0724/{}/{}_3p.png'.format(group, int(full_mass)), transparent=True)

In [ ]:
%run ../modules/cds.ipynb

In [ ]:
df1 = df_skeleton.copy()
df2 = df_gap_dots.copy()

# df1['Skeleton'] = True
# df2['Skeleton'] = False
df_merge = pd.concat([df1, df2, fullmass_dot]).drop_duplicates()
df_merge['Skeleton'] = False
df_merge.loc[df1.index, 'Skeleton'] = True
df_merge.loc[fullmass_dot.index, 'Skeleton'] = True
df_merge = df_merge.sort_values('Mass')

bcr = base_calling_random(df_merge, silence=True)
df_seats = process_mass_base_seats(df_merge, bcr[1], full_mass=full_mass, orientation=3)
df_seats.to_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/MassSum/II/Phe_MFE0724/{}/seats_{}_3p.xlsx'.format(group, int(full_mass)))
df_merge.to_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/MassSum/II/Phe_MFE0724/{}/{}_3p.xlsx'.format(group, int(full_mass)))

In [ ]:
def find_gap_dots(df, df_skeleton, fullmass_dot, orientation=5):
    TAG_5P = 79.9663
    TAG_3OH = 18.0106

    df_skeleton = df_skeleton.sort_values('Mass')

    idxs = list()
    gap_idxs = list()
    for idx, row in df_skeleton.iterrows():
        print('Processing forward {}'.format(row.Mass))
        columns = ['Mass', 'RT', 'Vol']
        df_ends = pd.DataFrame(index=[0, 1], columns=columns)
        if not idxs:
            # handle gap between TAG and the first dot
            if orientation == 3:
                pre = TAG_3OH - TAG_5P
            else:
                pre = TAG_5P + TAG_3OH
            df_ends.iloc[0] = [pre, 0.001, 0]
        else:
            pre = df_skeleton.loc[idxs[-1]].Mass
            df_ends.iloc[0] = df_skeleton.loc[idxs[-1]][columns]
        is_last = True if idx == df_skeleton.index[-1] else False
        if not is_last and row.Mass - pre < 600:
            continue
        if row.Mass - pre > 10000:
            idxs.append(idx)
            continue
        print('components {}-{}'.format(row.Mass, pre))
        res = components(row.Mass - pre)
        if not res.empty:
            # keep this dot, and fill in the gap
            print('keep this dot {}'.format(row.Mass))
            idxs.append(idx)
    #         continue
            df_ends.iloc[1] = row[columns]
    #         print(df_ends, df_ends.info())
            df_ends = df_ends.astype(float)
            df_alones = standalone_dots(df, df_ends, mode='all')
            if not df_alones.empty:
                print('extend dots ', df_alones)
                gap_idxs.extend(df_alones.index)
    #         print(df_alones)

        if is_last:
            df_ends.iloc[0] = df_skeleton.loc[idxs[-1]][columns]
            df_ends.iloc[1] = fullmass_dot.iloc[0][columns]
            if orientation == 5:
                df_ends.iloc[1]['Mass'] += TAG_5P
#             df_alones = standalone_dots(df_5p, df_ends, mode='all')
            df_alones = standalone_dots(df, df_ends, mode='all')
            if not df_alones.empty:
                print('extend dots ', df_alones)
                gap_idxs.extend(df_alones.index)

            # append the fullmass dot
            gap_idxs.append(fullmass_dot.index[0])

    print(gap_idxs)
    return df.loc[gap_idxs]

def find_gap_dots2(df, df_skeleton):
    TAG_5P = 79.9663
    TAG_3OH = 18.0106

    df_skeleton = df_skeleton.sort_values('Mass')
    idxs = list()
    gap_idxs = list()
    for idx, row in df_skeleton.iterrows():
        print('Processing {}'.format(row.Mass))
        columns = ['Mass', 'RT', 'Vol']
        df_ends = pd.DataFrame(index=[0, 1], columns=columns)
        if not idxs:
            # handle gap between TAG and the first dot
            pre = TAG_5P + TAG_3OH
            df_ends.iloc[0] = [pre, 0.001, 0]
    #         df_ends.iloc[0]['Mass'] = pre
    #         df_ends.iloc[0]['RT'] = 0
    #         df_ends.iloc[0]['Vol'] = 0
        else:
            pre = df_skeleton.loc[idxs[-1]].Mass
            df_ends.iloc[0] = df_skeleton.loc[idxs[-1]][columns]
        res = components(row.Mass - pre)
        if not res.empty:
            # keep this dot, and fill in the gap
            idxs.append(idx)

            df_ends.iloc[1] = row[columns]
            print(df_ends, df_ends.info())
            df_ends = df_ends.astype(float)
            df_alones = standalone_dots(df, df_ends)
            print(df_alones)
            gap_idxs.extend(list(df_alones.index))
    
    return df.loc[gap_idxs]

In [ ]:
full_mass = 24581.374101 #24189.23 #24245.225 #24941.525 #24252.315242 #24581.374101 
df_3p_24581, df_5p_24581 = computational_data_seperation(df_3p, df_5p, full_mass, ignore_endpoints=True)
plotly_zones(df_5p_24581, df_3p_24581)
df_24581 = pd.concat([df_3p_24581, df_5p_24581]).drop_duplicates()
df_24581.shape

In [ ]:
full_mass = 24252.31 #24189.23 #24245.225 #24941.525 #24252.315242 #24581.374101 
df_3p_24252, df_5p_24252 = computational_data_seperation(df_3p, df_5p, full_mass, ignore_endpoints=True)
plotly_zones(df_5p_24252, df_3p_24252)
df_24252 = pd.concat([df_3p_24252, df_5p_24252]).drop_duplicates()
df_24252.shape

In [ ]:
full_mass = 24596.36  #24189.23 #24245.225 #24941.525 #24252.315242 #24581.374101 
df_3p_24596, df_5p_24596 = computational_data_seperation(df_3p, df_5p, full_mass, ignore_endpoints=True)
plotly_zones(df_5p_24596, df_3p_24596)
df_24596 = pd.concat([df_3p_24596, df_5p_24596]).drop_duplicates()
df_24596.shape

In [ ]:
full_mass = 24267.31  #24189.23 #24245.225 #24941.525 #24252.315242 #24581.374101 
df_3p_24267, df_5p_24267 = computational_data_seperation(df_3p, df_5p, full_mass, ignore_endpoints=True)
plotly_zones(df_5p_24267, df_3p_24267)
df_24267 = pd.concat([df_3p_24267, df_5p_24267]).drop_duplicates()
df_24267.shape

In [ ]:
df_tmp = df_3p_24267.copy()
df_tmp.Mass += 329.0525
plotly_zones(df_3p_24596, df_tmp)

In [ ]:
full_mass = 24597.35  #24189.23 #24245.225 #24941.525 #24252.315242 #24581.374101 
df_3p_24597, df_5p_24597 = computational_data_seperation(df_3p, df_5p, full_mass, ignore_endpoints=True)
plotly_zones(df_5p_24597, df_3p_24597)
df_24597 = pd.concat([df_3p_24597, df_5p_24597]).drop_duplicates()
df_24597.shape

In [ ]:
full_mass = 24268.3  #24189.23 #24245.225 #24941.525 #24252.315242 #24581.374101 
df_3p_24268, df_5p_24268 = computational_data_seperation(df_3p, df_5p, full_mass, ignore_endpoints=True)
plotly_zone(df_3p_24268)
plotly_zone(df_5p_24268)
plotly_zones(df_3p_24268, df_5p_24268)
df_24268 = pd.concat([df_3p_24268, df_5p_24268]).drop_duplicates()
df_24268.shape

In [ ]:
full_mass = 25334.57 #24189.23 #24245.225 #24941.525 #24252.315242 #24581.374101 
df_3p_25334, df_5p_25334 = computational_data_seperation(df_3p, df_5p, full_mass, ignore_endpoints=True, error=0.15)
plotly_zone(df_3p_25334)
df_25334 = pd.concat([df_3p_25334, df_5p_25334]).drop_duplicates()
df_25334.shape

In [ ]:
full_mass = 25005.52 #24189.23 #24245.225 #24941.525 #24252.315242 #24581.374101 
df_3p_25005, df_5p_25005 = computational_data_seperation(df_3p, df_5p, full_mass, ignore_endpoints=True, error=0.15)
plotly_zones(df_5p_25005, df_3p_25005)
df_25005 = pd.concat([df_3p_25005, df_5p_25005]).drop_duplicates()
df_25005.shape

In [ ]:
tmp = df_3p_25005.copy()
tmp.Mass += 329.0524
plotly_zones(tmp, df_3p_25334)

In [ ]:
# plotly_zones(df_5p_25005, df_5p_25334)
df_common = df_5p_25005[df_5p_25005.Mass.isin(df_5p_25334.Mass)]
plotly_multi_zones([df_5p_25005, df_5p_25334, df_common])

In [ ]:
full_mass = 24332.31#24661.36 #24189.23 #24245.225 #24941.525 #24252.315242 #24581.374101 
df_3p_24332, df_5p_24332 = computational_data_seperation(df_3p, df_5p, full_mass, ignore_endpoints=True)
plotly_zones(df_5p_24332, df_3p_24332)
df_24332 = pd.concat([df_3p_24332, df_5p_24332]).drop_duplicates()
df_24332.shape

In [ ]:
full_mass = 24027.24#24661.36 #24189.23 #24245.225 #24941.525 #24252.315242 #24581.374101 
df_3p_24027, df_5p_24027 = computational_data_seperation(df_3p, df_5p, full_mass, ignore_endpoints=True)
plotly_zones(df_5p_24027, df_3p_24027)
df_24027 = pd.concat([df_3p_24027, df_5p_24027]).drop_duplicates()
df_24027.shape

In [ ]:
df_common = df_5p_24332[df_5p_24332.Mass.isin(df_5p_24027.Mass)]
# plotly_multi_zones([df_5p_24332, df_5p_24027, df_common])
plotly_zone(df_common)

In [ ]:
full_mass = 24445.35 #24116.27 ##24661.36 #24189.23 #24245.225 #24941.525 #24252.315242 #24581.374101 
df_3p_24445, df_5p_24445 = computational_data_seperation(df_3p, df_5p, full_mass, ignore_endpoints=True)
plotly_zones(df_5p_24445, df_3p_24445)
df_24445 = pd.concat([df_3p_24445, df_5p_24445]).drop_duplicates()
df_24445.shape

In [ ]:
full_mass = 24116.27 # ##24661.36 #24189.23 #24245.225 #24941.525 #24252.315242 #24581.374101 
df_3p_24116, df_5p_24116 = computational_data_seperation(df_3p, df_5p, full_mass, ignore_endpoints=True)
plotly_zones(df_5p_24116, df_3p_24116)
df_24116 = pd.concat([df_3p_24116, df_5p_24116]).drop_duplicates()
df_24116.shape

In [ ]:
full_mass = 24668.41 #24997.46 #23722.165
df_3p_no, df_5p_no = computational_data_seperation(df_3p, df_5p, full_mass, ignore_endpoints=True)
plotly_zones(df_5p_no, df_3p_no)
df_no = pd.concat([df_3p_no, df_5p_no]).drop_duplicates()
df_no.shape

In [ ]:
df_merge = pd.concat([df_5p_24332, df_5p_24027]).drop_duplicates()
bcr = base_calling_random(df_merge, silence=True)
plotly_basecalling(*bcr)
df_seats = process_mass_base_seats(df_merge, bcr[1], full_mass=24332.31)
df_seats.shape
df_seats.to_excel('/Users/bryan/Documents/Presentations/Feb 3 2021/seats.xlsx')

In [ ]:
df_3p.shape

In [ ]:
# plotly_zones(df_3p_24581, df_3p_24252)
# df_tmp = df_3p_24252.copy()
# df_tmp.Mass = df_tmp.Mass + 329.0525
# plotly_zones(df_3p_24581, df_tmp)

# plotly_zones(df_5p_24416, df_5p_24087)
# plotly_zones(df_5p_24581, df_5p_24416)
# plotly_zones(df_5p_24252, df_5p_24087)
# plotly_zones(df_24252, df_24087)
# plotly_zones(df_3p_24581, df_3p_24416)

# plotly_zones(df_3p_24252, df_3p_24087)
# df_tmp = df_3p_24087.copy()
# df_tmp.Mass = df_tmp.Mass + 165.07
# plotly_zones(df_3p_24252, df_tmp)

# # plotly_zones(df_3p_24581, df_3p_24597)
# df_tmp = df_3p_24597.copy()
# # df_tmp.Mass = df_tmp.Mass - 16
# plotly_zones(df_3p_24581, df_tmp)

# plotly_zones(df_5p_24581, df_5p_24252, y='RT')
# plotly_zones(df_5p_24332, df_5p_24027, y='RT')
# plotly_zone(df_3p_24332)
# plotly_zone(df_3p_24027)
plotly_zone(df_3p_24596)
plotly_zone(df_3p_24267)

# plotly_zones(df_3p_24252, df_3p_24268)
# plotly_zones(df_3p_24581, df_3p_24597)
# plotly_zones(df_24252, df_24268)
# plotly_zones(df_24252, df_24597)
# plotly_multi_zones([df_5p_24581, df_24597, df_24252, df_24268])

In [ ]:
df_left_gap = dots_in_left_gap(df_5p, 2826.404, orientation=5)
df_left_gap.shape

In [ ]:
# df_mid_gap = dots_in_mid_gap(df_5p, 2548.3381, 3856.531)
df_mid_gap = dots_in_mid_gap(df_5p, 9535.374254, 14310.00116)
df_mid_gap.shape

In [ ]:
# df_right_gap = dots_in_right_gap(df_phe, 20449.84227, 24267.31, orientation=3)
df_right_gap = dots_in_right_gap(df_phe, 21150.87924, 24596.36, orientation=5)
df_right_gap.shape

In [ ]:
%run ../modules/utils.ipynb
components(674.1275284)

In [ ]:

# plotly_multi_zones([df_3p_24332, df_mid_gap])
# plotly_multi_zones([df_3p_24027, df_mid_gap])
df_3p_tmp = df_3p_24332.copy()
df_3p_tmp.Mass = df_3p_tmp.Mass - 305.0413
# plotly_multi_zones([df_3p_24332, df_3p_tmp])

df_common = match_dfs(df_3p_tmp, df_3p_24027)
# plotly_zone(df_common)
# plotly_multi_zones([df_3p_24027, df_left_gap[df_left_gap.Hit > 3]])
# plotly_multi_zones([df_3p_24027, df_mid_gap, df_left_gap])
# plotly_multi_zones([df_3p_24267, df_mid_gap, df_left_gap, df_right_gap])
plotly_multi_zones([df_5p_25334,  df_mid_gap,])
# df_3p_tmp = df_3p_24252.copy()
# df_3p_tmp.Mass = df_3p_tmp.Mass + 329.0525
# plotly_multi_zones([df_3p_24581, df_3p_tmp])

# plotly_multi_zones([df_5p_24332, df_mid_gap, df_right_gap])
# plotly_multi_zones([df_5p_24416, df_5p_24087, df_left_gap])

In [ ]:
df_left_gap = mutual_connection(df_left_gap)

In [ ]:
import random

def mutual_connection(_df):
    df = _df.copy()
    df['Hit'] = 0
    len = df.shape[0]
    for i in range(len):
        print('Processing {}'.format(df.iloc[i].Mass))
        count = 0
        j_pool = set()
        N = 5 if i > 3 else 2
        while count < N:
            j = random.randint(0, len-1)
            if j == i or j in j_pool:
                continue
    #         print('i {} j {}, len {}'.format(i, j, len))
            diff = abs(df.iloc[i].Mass - df.iloc[j].Mass)
            if diff < 400 or diff > 3000:
                continue

            j_pool.add(j)

            print('--handle Mass {}'.format(df.iloc[j].Mass))
            df_condidates = components(diff)
            print(df_condidates)
            if not df_condidates.empty:
                print('Hit index {}.'.format(i))
                df['Hit'].iloc[i] += 1

            count += 1

    print(df)
    return df
    

In [ ]:

df_mid_ends = pd.DataFrame()
df_mid_ends['Mass'] = np.array([9535.374254, 14310.00116])
df_mid_ends = match_dfs(df_mid_ends, df_5p)
df_tmp = pd.concat([df_mid_gap, df_mid_ends])
bcr = base_calling_random(df_tmp, silence=True)
plotly_basecalling(*bcr)

In [ ]:
df_3p_24267.to_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/MassSum/Phe_MFE0724/Group_24596/skeleton.xlsx')

In [ ]:
df_merge = df_3p_24267 #pd.concat([df_5p_24332, df_5p_24027]).drop_duplicates()
bcr = base_calling_random(df_merge, silence=True)
df_seats = process_mass_base_seats(df_merge, bcr[1], full_mass=24267.31)
df_seats.to_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/MassSum/Phe_MFE0724/Group_24596/seats.xlsx')

In [ ]:
df_left_gap = dots_in_left_gap(df_3p, 2789.406, orientation=3)
df_left_gap.shape

In [ ]:
plot_zones(df_3p_24581, df_left_gap)
plotly_zone(df_left_gap, y='Vol')

In [ ]:
%run ../modules/utils.ipynb

In [ ]:
df_seats = process_mass_base_seats(df_merge, bcr[1], full_mass=24581.38)
df_seats.to_excel('/Users/bryan/Documents/Presentations/Feb 3 2021/seats.xlsx')

In [ ]:
df_merge = pd.concat([df_5p_24597, df_5p_24268]).drop_duplicates()
bcr = base_calling_random(df_merge) #[df_3p_cc.Mass>20000]
plotly_basecalling(*bcr, df_ori=df_merge)

In [ ]:
TAG_5P = 79.9663
TAG_3OH = 18.0106

def dots_in_left_gap(df, mass, orientation=5):
    columns = ['Mass', 'RT']
    df_ends = pd.DataFrame(index=[0, 1], columns=columns, dtype=float)
    left_mass = TAG_5P
    if orientation == 3:
        left_mass = TAG_3OH - TAG_5P
    df_ends.loc[0][columns] = [left_mass, 0.0]
    df_ends.loc[1][columns] = df[(df.Mass < mass+0.1) & (df.Mass > mass-0.1)].iloc[0][columns]
    df_gap = standalone_dots(df, df_ends, mode='all')
    return df_gap

def dots_in_right_gap(df, mass, full_mass, orientation=5):
    columns = ['Mass', 'RT']
    df_ends = pd.DataFrame(index=[0, 1], columns=columns, dtype=float)
    df_ends.loc[0][columns] = df[(df.Mass < mass+0.1) & (df.Mass > mass-0.1)].iloc[0][columns]
    df_ends.loc[1][columns] = df[(df.Mass < full_mass+0.1) & (df.Mass > full_mass-0.1)].iloc[0][columns]
    if orientation == 5:
        df_ends.loc[1]['Mass'] += TAG_5P
    df_gap = standalone_dots(df, df_ends, mode='all')
    return df_gap

def dots_in_mid_gap(df, left_mass, right_mass):
    columns = ['Mass', 'RT']
    df_ends = pd.DataFrame(index=[0, 1], columns=columns, dtype=float)
    df_ends.loc[0][columns] = df[(df.Mass < left_mass+0.1) & (df.Mass > left_mass-0.1)].iloc[0][columns]
    df_ends.loc[1][columns] = df[(df.Mass < right_mass+0.1) & (df.Mass > right_mass-0.1)].iloc[0][columns]
    df_gap = standalone_dots(df, df_ends, mode='all')
    return df_gap

In [ ]:
TAG_5P = 79.9663
TAG_3OH = 18.0106

start_mass = 20821.81608
full_mass = 24416.29
columns = ['Mass', 'RT']
df_ends = pd.DataFrame(index=[0, 1], columns=columns, dtype=float)
# df_ends.loc[0][columns] = df_5p[(df_5p.Mass < 21825.96+0.1) & (df_5p.Mass > 21825.96-0.1)].iloc[0][columns]
df_ends.loc[0][columns] = df_5p[(df_5p.Mass < start_mass+0.1) & (df_5p.Mass > start_mass-0.1)].iloc[0][columns]
df_ends.loc[1][columns] = df_5p[(df_5p.Mass < full_mass+0.1) & (df_5p.Mass > full_mass-0.1)].iloc[0][columns]
df_ends.loc[1]['Mass'] += TAG_5P
df_ends
df_alones_5p = standalone_dots(df_5p, df_ends, mode='all')
plotly_zone(df_alones_5p)

In [ ]:
df_alones_5p

In [ ]:
# df_alones_5p.append(df_5p[(df_5p.Mass < full_mass+0.1) & (df_5p.Mass > full_mass-0.1)].iloc[0])
# df_alones_5p
bcr = base_calling_random(df_alones_5p) #[df_3p_cc.Mass>20000]
plt, _ = plotly_basecalling(*bcr)

In [ ]:
tmp_mass = 283.0917 #588.133 #893.174299
df_3p[(df_3p.Mass < tmp_mass+0.1) & (df_3p.Mass > tmp_mass-0.1)]

In [ ]:
df_alones_5p.iloc[1]['Mass'] += TAG_5P

In [ ]:
columns = ['Mass', 'RT']
df_ends = pd.DataFrame(index=[0, 1], columns=columns, dtype=float)
# df_ends.loc[0][columns] = [-61.9557, 0.0]
df_ends.loc[0][columns] = [893.174299, 0.0]
df_ends.loc[1][columns] = df_24597[(df_24597.Mass < 2789.41) & (df_24597.Mass > 2789.40)].iloc[0][columns]
df_alones_3p = standalone_dots(df_3p, df_ends, mode='all')
df_alones_3p.shape

In [ ]:
df_alones_3p

In [ ]:
plotly_zone(df_alones_3p, y='Vol')

In [ ]:
plotly_zones(df_24597, df_alones_3p)
# plotly_zones(df_24268, df_alones_3p)

In [ ]:
bcr = base_calling_random(df_alones_3p, silence=True)
plotly_basecalling(*bcr)

In [ ]:
plotly_zone(df_5p_cc)

In [ ]:
bcr = base_calling_random(df_5p_cc) #[df_3p_cc.Mass>20000]
plt, _ = plot_basecalling(*bcr, annotate=False)

In [ ]:
fullmass_dot = df_phe[(df_phe.Mass>full_mass-0.1) & (df_phe.Mass<full_mass+0.1)]
fullmass_dot

In [ ]:
TAG_5P = 79.9663
TAG_3OH = 18.0106

df_sample = df_5p_cc #df_5p_cc_ref
df_sample = df_sample.sort_values('Mass')

idxs = list()
gap_idxs = list()
for idx, row in df_sample.iterrows():
    print('Processing forward {}'.format(row.Mass))
    columns = ['Mass', 'RT', 'Vol']
    df_ends = pd.DataFrame(index=[0, 1], columns=columns)
    if not idxs:
        # handle gap between TAG and the first dot
        pre = TAG_5P + TAG_3OH
        df_ends.iloc[0] = [pre, 0.001, 0]
    else:
        pre = df_sample.loc[idxs[-1]].Mass
        df_ends.iloc[0] = df_sample.loc[idxs[-1]][columns]
    is_last = True if idx == df_sample.index[-1] else False
    if not is_last and row.Mass - pre < 600:
        continue
    print('components {}-{}'.format(row.Mass, pre))
    res = components(row.Mass - pre)
    if not res.empty:
        # keep this dot, and fill in the gap
        print('keep this dot {}'.format(row.Mass))
        idxs.append(idx)
#         continue
        df_ends.iloc[1] = row[columns]
#         print(df_ends, df_ends.info())
        df_ends = df_ends.astype(float)
        df_alones = standalone_dots(df_5p, df_ends, mode='all')
        if not df_alones.empty:
            print('extend dots ', df_alones)
            gap_idxs.extend(df_alones.index)
#         print(df_alones)

    if is_last:
        df_ends.iloc[0] = df_sample.loc[idxs[-1]][columns]
        df_ends.iloc[1] = fullmass_dot.iloc[0][columns]
        df_ends.iloc[1]['Mass'] += TAG_5P
        df_alones = standalone_dots(df_5p, df_ends, mode='all')
        if not df_alones.empty:
            print('extend dots ', df_alones)
            gap_idxs.extend(df_alones.index)
        
        # append the fullmass dot
        gap_idxs.append(fullmass_dot.index[0])
        
print(gap_idxs)
print(df_5p.loc[gap_idxs])

In [ ]:
components(918.0750643431002)

In [ ]:
df_gap_dots = df_5p.loc[gap_idxs]
df_gap_dots

In [ ]:
plot_zones(df_gap_dots, df_sample)

In [ ]:
df_5p_cc.to_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/MassSum/Yue/24203.24/skeleton.xlsx')
# df_3p_cc.to_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/MassSum/Yue/24941.525/skeleton_3p.xlsx')

In [ ]:
df_5p_cc.to_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/MassSum/Yue/25334.618/skeleton_5p_top.xlsx')
df_3p_cc.to_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/MassSum/Yue/25334.618/skeleton_3p_top.xlsx')

In [ ]:
df_all = pd.concat([df_gap_dots, df_sample])
df_all.drop_duplicates().to_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/MassSum/Yue/25334.618/skeleton_with_dots_in_gap_5p.xlsx')

In [ ]:
fpath = '/Users/bryan/Documents/Presentations/Feb 3 2021/seats_25334.csv'
df1 = pd.read_csv(fpath)
plt = plot_zone(df1)
png = fpath.split('.')[0]+'.png'
plt.savefig(png, transparent=True)